<a href="https://colab.research.google.com/github/IEXE-Tec/computo-datos-masivos/blob/master/04_emr_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>